# In this notebook, airports information has been gathered using different ICAO codes, then a dataframe has been created and information filtered to upload the resulting dataframe to MySQL

In [3]:
import pandas as pd
import requests

In [6]:
madrid_ICAO = 'LEMD'
barcelona_ICAO = 'LEBL'
malaga_ICAO = 'LEMG'
berlin_ICAO = 'EDDB'
harmburg_ICAO = 'EDDH'
london_ICAO = 'EGLL'

# This function will gather the necessary information from the aerobox API and store it in a dataframe called flights

def flight_arrivals_by_airport(icao_code):
  url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_code}/2022-10-16T20:00/2022-10-17T08:00"

  querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}

  headers = {
    "X-RapidAPI-Key": "My_key",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }
  response = requests.request("GET", url, headers=headers, params=querystring)  
  flight_arrivals = pd.json_normalize(response.json()['arrivals'])
  flights =(flight_arrivals.filter(['number','departure.airport.icao', 'departure.airport.name', 'departure.scheduledTimeLocal','departure.actualTimeLocal', 'arrival.scheduledTimeLocal', 'arrival.actualTimeLocal','airline.name']))
  flights.insert(1, "arrival_ICAO", icao_code, True)
  return flights

In [7]:
flights = flight_arrivals_by_airport(barcelona_ICAO)

In [117]:
# Adding a column at 0 position for flights id
flights.insert(0,'flight_id',flights['number'])

In [41]:
flights

,flight_id,number,departure.airport.icao,departure.airport.name,departure.scheduledTimeLocal,departure.actualTimeLocal,arrival.scheduledTimeLocal,arrival.actualTimeLocal,airline.name
0,3O 375,3O 375,GMMW,Nador,2022-10-15 17:35+01:00,NaN,2022-10-15 20:10+02:00,2022-10-15 20:10+02:00,Air Arabia Maroc
1,AA 8857,AA 8857,LEZL,Seville,2022-10-15 18:25+02:00,2022-10-15 18:25+02:00,2022-10-15 20:05+02:00,2022-10-15 20:05+02:00,American
2,AA 8871,AA 8871,LIRQ,Firenze,NaN,NaN,2022-10-15 20:50+02:00,2022-10-15 20:50+02:00,American
3,AY 1653,AY 1653,EFHK,Helsinki,2022-10-15 17:05+03:00,2022-10-15 17:05+03:00,2022-10-15 20:05+02:00,2022-10-15 20:05+02:00,Finnair
4,BA 476,BA 476,EGLL,London,2022-10-15 17:20+01:00,2022-10-15 17:20+01:00,2022-10-15 20:30+02:00,2022-10-15 20:30+02:00,British Airways
...,...,...,...,...,...,...,...,...,...
179,FR 6363,FR 6363,LEPA,Palma De Mallorca,2022-10-16 06:30+02:00,NaN,2022-10-16 07:25+02:00,NaN,Ryanair
180,QR 137,QR 137,OTHH,Doha,2022-10-16 01:30+03:00,NaN,2022-10-16 07:30+02:00,NaN,Qatar Airways
181,TO 4750,TO 4750,LFPO,Paris,2022-10-16 06:00+02:00,NaN,2022-10-16 07:40+02:00,NaN,Transavia France
182,UX 6006,UX 6006,LEPA,Palma De Mallorca,2022-10-16 07:05+02:00,NaN,2022-10-16 07:55+02:00,NaN,Air Europa


In [137]:
# Changing the name of the columns so they match the tables in MySQL.

flights.columns = ['flight_id','arrival_ICAO','departure_ICAO','departure_name','departure_sch_time','departure_actual_time','arrival_sch_time','arrival_actual_time','airline_name']

In [138]:
flights

,flight_id,arrival_ICAO,departure_ICAO,departure_name,departure_sch_time,departure_actual_time,arrival_sch_time,arrival_actual_time,airline_name
0,AA 8857,LEBL,LEZL,Seville,2022-10-16 18:50+02:00,2022-10-16 18:50+02:00,2022-10-16 20:30+02:00,2022-10-16 20:30+02:00,American
1,AY 1653,LEBL,EFHK,Helsinki,2022-10-16 17:05+03:00,2022-10-16 17:05+03:00,2022-10-16 20:05+02:00,2022-10-16 20:05+02:00,Finnair
2,BA 488,LEBL,EGLL,London,2022-10-16 17:05+01:00,2022-10-16 17:05+01:00,2022-10-16 20:15+02:00,2022-10-16 20:15+02:00,British Airways
3,D8 3658,LEBL,EKCH,Copenhagen,2022-10-16 17:05+02:00,2022-10-16 17:05+02:00,2022-10-16 20:10+02:00,2022-10-16 20:10+02:00,Norwegian Air International
4,EC 2761,LEBL,LIMC,Milan,NaN,NaN,2022-10-16 20:30+02:00,2022-10-16 20:30+02:00,EasyJet Europe
...,...,...,...,...,...,...,...,...,...
203,FR 9369,LEBL,LIPE,Bologna,2022-10-17 06:05+02:00,NaN,2022-10-17 07:50+02:00,NaN,Ryanair
204,QR 137,LEBL,OTHH,Doha,2022-10-17 01:30+03:00,NaN,2022-10-17 07:30+02:00,NaN,Qatar Airways
205,TO 4750,LEBL,LFPO,Paris,2022-10-17 06:00+02:00,NaN,2022-10-17 07:40+02:00,NaN,Transavia France
206,UX 6006,LEBL,LEPA,Palma De Mallorca,2022-10-17 07:05+02:00,NaN,2022-10-17 07:55+02:00,NaN,Air Europa


In [139]:
# Verifying the datatypes, which are all objects, but this is not the case so they need to be assigned the correct type.

flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flight_id              208 non-null    object
 1   arrival_ICAO           208 non-null    object
 2   departure_ICAO         208 non-null    object
 3   departure_name         208 non-null    object
 4   departure_sch_time     176 non-null    object
 5   departure_actual_time  114 non-null    object
 6   arrival_sch_time       208 non-null    object
 7   arrival_actual_time    170 non-null    object
 8   airline_name           208 non-null    object
dtypes: object(9)
memory usage: 14.8+ KB


In [140]:
flights['departure_sch_time'] = pd.to_datetime(flights['departure_sch_time'], utc=True)

In [141]:
flights['departure_actual_time'] = pd.to_datetime(flights['departure_actual_time'], utc=True)

In [142]:
flights['arrival_sch_time'] = pd.to_datetime(flights['arrival_sch_time'], utc=True)

In [143]:
flights['arrival_actual_time'] = pd.to_datetime(flights['arrival_actual_time'], utc=True)

In [144]:
# After assigning the correct dataframes, another look to see if everything is correct.

flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   flight_id              208 non-null    object             
 1   arrival_ICAO           208 non-null    object             
 2   departure_ICAO         208 non-null    object             
 3   departure_name         208 non-null    object             
 4   departure_sch_time     176 non-null    datetime64[ns, UTC]
 5   departure_actual_time  114 non-null    datetime64[ns, UTC]
 6   arrival_sch_time       208 non-null    datetime64[ns, UTC]
 7   arrival_actual_time    170 non-null    datetime64[ns, UTC]
 8   airline_name           208 non-null    object             
dtypes: datetime64[ns, UTC](4), object(5)
memory usage: 14.8+ KB


In [148]:
# And here the dataframe will be uploaded to MySQL.

import sqlalchemy
import pymysql

In [149]:
schema="GansWeatherDB"
host="127.0.0.1"
user="My_user"
password="My_pass"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [150]:
flights.dropna().to_sql('flights', if_exists='append', con=con, index=False)

In [81]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LEBL/2022-10-15T20:00/2022-10-16T08:00"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}

headers = {
"X-RapidAPI-Key": "My_key",
"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}
response = requests.request("GET", url, headers=headers, params=querystring)  
flight_arrivals = pd.json_normalize(response.json()['departures'])

In [114]:
flights

,arrival_ICAO,flight_id,flight_number,departure_ICAO,departure_name,departure_sch_time,departure_actual_time,arrival_sch_time,arrival_actual_time,airline_name
0,3O 375,LEBL,3O 375,GMMW,Nador,2022-10-15 16:35:00+00:00,NaT,2022-10-15 18:10:00+00:00,2022-10-15 18:10:00+00:00,Air Arabia Maroc
1,AA 8857,LEBL,AA 8857,LEZL,Seville,2022-10-15 16:25:00+00:00,2022-10-15 16:25:00+00:00,2022-10-15 18:05:00+00:00,2022-10-15 18:05:00+00:00,American
2,AA 8871,LEBL,AA 8871,LIRQ,Firenze,NaT,NaT,2022-10-15 18:50:00+00:00,2022-10-15 18:50:00+00:00,American
3,AY 1653,LEBL,AY 1653,EFHK,Helsinki,2022-10-15 14:05:00+00:00,2022-10-15 14:05:00+00:00,2022-10-15 18:05:00+00:00,2022-10-15 18:05:00+00:00,Finnair
4,BA 476,LEBL,BA 476,EGLL,London,2022-10-15 16:20:00+00:00,2022-10-15 16:20:00+00:00,2022-10-15 18:30:00+00:00,2022-10-15 18:30:00+00:00,British Airways
...,...,...,...,...,...,...,...,...,...,...
244,TO 4750,LEBL,TO 4750,LFPO,Paris,2022-10-16 04:00:00+00:00,NaT,2022-10-16 05:40:00+00:00,2022-10-16 05:40:00+00:00,Transavia France
245,QR 3655,LEBL,QR 3655,LEPA,Palma De Mallorca,2022-10-16 04:50:00+00:00,2022-10-16 04:50:00+00:00,2022-10-16 05:45:00+00:00,2022-10-16 05:45:00+00:00,Qatar Airways
246,UL 3547,LEBL,UL 3547,OTHH,Doha,NaT,NaT,2022-10-16 05:30:00+00:00,2022-10-16 05:30:00+00:00,SriLankan
247,UX 6006,LEBL,UX 6006,LEPA,Palma De Mallorca,2022-10-16 05:05:00+00:00,2022-10-16 05:05:00+00:00,2022-10-16 05:55:00+00:00,2022-10-16 05:55:00+00:00,Air Europa
